**PREDICTING PULSARS BASED ON STATISTICS OF DISTRIBUTION OF EMISSIONS PATTERNS**

**Introduction:**


**Preliminary exploratory data analysis:**

In [55]:
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)
temp <- tempfile()
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip",temp)
pulsar_data <- read.csv(unz(temp, "HTRU_2.csv"), header = FALSE)
unlink(temp)
colnames(pulsar_data) <- c("mean_of_ip", "sd_of_ip", "kurt_of_ip", "skew_of_ip", 
                           "mean_of_DM-SNR", "sd_of_DM-SNR", "kurt_of_DM-SNR", "skew_of_DM-SNR", "class")
pulsar_data <- pulsar_data %>%
    mutate(class = as.factor(class))
head(pulsar_data)

,mean_of_ip,sd_of_ip,kurt_of_ip,skew_of_ip,mean_of_DM-SNR,sd_of_DM-SNR,kurt_of_DM-SNR,skew_of_DM-SNR,class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,140.56250,55.68378,-0.23457141,-0.6996484,3.199833,19.11043,7.975532,74.24222,0
2,102.50781,58.88243,0.46531815,-0.5150879,1.677258,14.86015,10.576487,127.39358,0
3,103.01562,39.34165,0.32332837,1.0511644,3.121237,21.74467,7.735822,63.17191,0
4,136.75000,57.17845,-0.06841464,-0.6362384,3.642977,20.95928,6.896499,53.59366,0
5,88.72656,40.67223,0.60086608,1.1234917,1.178930,11.46872,14.269573,252.56731,0
6,93.57031,46.69811,0.53190485,0.4167211,1.636288,14.54507,10.621748,131.39400,0
